# Data Cleaning of Movie Dataset

This notebook cleans and prepares movies dataset by addressing data quality issues such as missing values, duplicate entries, and data inconsistencies. 
The goal is to create a high-quality dataset for accurate analysis and modeling.

Files cleaned:
- movies_metadata.csv: The main Movies Metadata file. Contains information on 45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, revenue, release dates, languages, production countries and companies.
- keywords.csv: Contains the movie plot keywords for our MovieLens movies. Available in the form of a stringified JSON Object.
- credits.csv: Consists of Cast and Crew Information for all our movies. Available in the form of a stringified JSON Object.


Data Source: https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset/data

In [1]:
import pandas as pd
from pandas import json_normalize
import ast
import numpy as np


# Show full content in each cell
#pd.set_option('display.max_colwidth', None)
#pd.set_option('display.max_rows', None)  # Show all rows
#pd.set_option('display.max_columns', None)  # Show all columns
# Reset the display options to their default values
pd.reset_option('display.max_rows')
pd.reset_option('display.max_columns')
pd.reset_option('display.max_colwidth')

### Functions

In [2]:
def must_be_type(value, c_type):
    """
    Function: Standardizes values in a column to match a specified data type.

    Parameters:
    value (str): A string representing the data (e.g., text, numbers, lists, JSON, etc.).
    c_type (str): Desired data type for the value:
                  - 'text': Convert to plain text; non-text values (e.g., lists, dicts, numbers) become NaN.
                  - 'numeric': Convert to a number (int/float); incompatible values become NaN.
                  - 'json': Convert to a JSON object (list/dict); invalid types become NaN.
                  - 'boolean': Convert to boolean; incompatible values become NaN.
                  - 'date': Convert to datetime; invalid formats become NaN.

    Returns:
    The value converted to the specified type, or NaN if the type conversion is not possible.
    """
    # Attempt to parse the value in string format.
    try:
        # To convert string values into: lists, dicts, tuples, int, float, bool, str.
        parsed_value = ast.literal_eval(value)
    except (ValueError, SyntaxError):
        # If parsing fails, keep the original value
        parsed_value = value

    # Handle conversion based on desired data type
    if c_type == 'text':
        # For text: Return NaN if value is a complex type like list, dict, int, or float
        if isinstance(parsed_value, (list, dict, int, float, bool)):
            return np.nan
        else:
            return parsed_value

    elif c_type == 'numeric':
        # For numeric: Return NaN if value is not int or float
        if isinstance(parsed_value, (int, float)):
            return pd.to_numeric(parsed_value, errors='coerce')
        else:
            return np.nan

    elif c_type == 'json':
        # For JSON: Return NaN if value is not JSON style list or dict value
        if isinstance(parsed_value, (list, dict)):
            return parsed_value
        else:
            return np.nan
        
    elif c_type == 'boolean':
        # For boolean: Return NaN if not a bool value
        if isinstance(parsed_value, (bool)):
            return parsed_value
        else:
            return np.nan
        
    elif c_type == 'date':
        # For date: Return converted datetime, if can't be converted return NaN
        parsed_value = pd.to_datetime(parsed_value, errors='coerce')
        return parsed_value

    # Return nan if no conditions are met
    return np.nan


In [3]:
def extract_val_json(value, name):

    '''
    Function: Extract specified field values from a JSON-like list and return them as a single comma-separated string.
    
    Parameters:
    value: A list or dictionary containing structured data (e.g., genres, cast, etc.).
    name: The specific field name to extract from each item in the list (e.g., 'name', 'id').

    Returns: A comma-separated string of the extracted values if the list is not empty. Returns NaN if the list is empty, the input is not a list or dictionary, or the specified field does not exist.
    '''    
    if isinstance(value, (list, dict)):
        if len(value) > 0:
            return ', '.join(json_normalize(value)[name])
        else:
            return np.nan
    else:
        return np.nan

## Table 1: movies_metadata.csv

In [4]:
# Load the csv file
df = pd.read_csv('original_files/movies_metadata.csv')
df.head(5) 

/var/folders/3_/ft87w8295r1_dk2lx99fbc0h0000gn/T/ipykernel_17053/1119314404.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('original_files/movies_metadata.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [ ]:
'''
Observations:
There are multiple columns where data is improperly formatted or filled incorrectly.
Examples:
1. String Boolean Values: The `df['adult']` column has `'True' and 'False'` stored as strings, but it also contains random text like movie descriptions, indicating misalignment.
2. String Numeric Values: The `df['budget']` column has numeric strings like '50000' but also includes inconsistent entries like file paths `("/ff9qCepilowshEtG2GYWwzt2bs4.jpg")`.

Next Steps:
Use the custom `must_be_type()` function to check and convert each column to the appropriate data type. Unexpected values will be replaced with NaN to ensure consistency and prevent errors during analysis.
'''

In [6]:
# Incorrect data found
df['adult'].value_counts()

adult
False                                                                                                                             45454
True                                                                                                                                  9
 - Written by Ørnås                                                                                                                   1
 Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.                        1
 Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.        1
Name: count, dtype: int64

In [35]:
# Incorrect data found
try:
    pd.to_numeric(df['budget'])
except ValueError as e:
    print(f"Error: {e}")

Error: Unable to parse string "/ff9qCepilowshEtG2GYWwzt2bs4.jpg" at position 19730


### Change data types, ensure no incorrect data is filled

In [8]:
# Create a copy of the original DataFrame to store the cleaned data
movies_metadata = df.copy()

# Define the expected data types for each column
column_type_mapping = {
    'adult': 'boolean',
    'belongs_to_collection': 'json',
    'budget': 'numeric',
    'genres': 'json',
    'homepage': 'text',
    'id': 'numeric',
    'imdb_id': 'text',
    'original_language': 'text',
    'original_title': 'text',
    'overview': 'text',
    'popularity': 'numeric',
    'poster_path': 'text',
    'production_companies': 'json',
    'production_countries': 'json',
    'release_date': 'date',
    'revenue': 'numeric',
    'runtime': 'numeric',
    'spoken_languages': 'json',
    'status': 'text',
    'title': 'text',
    'tagline': 'text',
    'video': 'boolean',
    'vote_average': 'numeric',
    'vote_count': 'numeric'
}
# Apply must_be_type() function to each column based on the mapping
for value, c_type in column_type_mapping.items():
    movies_metadata[value] = df[value].apply(lambda x: must_be_type(x, c_type))

<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal
<unknown>:1: SyntaxWarning: invalid decimal literal


In [9]:
# Display 5 random rows of the cleaned DataFrame
movies_metadata.sample(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
9124,False,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,35016.0,tt0121442,nl,Koko Flanel,Placide's dad tells him on his deathbed he'll ...,...,1990-02-23,0.0,95.0,"[{'iso_639_1': 'nl', 'name': 'Nederlands'}]",Released,Fashion isn't everything,Koko Flanel,False,5.7,11.0
37274,False,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,295782.0,tt1256505,de,"Die Frau, die im Wald verschwand",NaN,...,2009-04-29,0.0,NaN,[],Released,NaN,"Die Frau, die im Wald verschwand",False,0.0,0.0
33579,False,"{'id': 476054, 'name': 'The Hitcher (1986) Col...",0.0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,20381.0,tt0299988,en,The Hitcher II: I've Been Waiting,A sadistic serial killer terrorizes a couple d...,...,2003-07-15,0.0,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,The Hitcher II: I've Been Waiting,False,4.3,19.0
18150,False,"{'id': 33514, 'name': 'The Twilight Collection...",110000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://www.breakingdawn-themovie.com/,50619.0,tt1324999,en,The Twilight Saga: Breaking Dawn - Part 1,The new found married bliss of Bella Swan and ...,...,2011-03-15,712171856.0,117.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Forever is just the beginning,The Twilight Saga: Breaking Dawn - Part 1,False,5.8,2622.0
45006,False,NaN,0.0,"[{'id': 99, 'name': 'Documentary'}]",http://www.exhibitapictures.com,359749.0,tt4372240,en,78/52,78 shots. 52 cuts. The shower scene from PSYCHO.,...,2017-11-03,0.0,91.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,78/52,False,0.0,4.0


### Extract important fields from JSON like data

In [ ]:
'''
Several columns in the dataset contain nested JSON structures (e.g., 'genres', 'production_companies'). 
Custom function, extract_val_json() has been created to extract relavent fields e.g. 'name' and transform the results into a readable format. 
This step ensures that each column only retains the relevant information for further analysis.
'''

In [10]:
# Apply the extract_val_json function to multiple columns with JSON-like structures.
json_columns = ['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages']

# These columns have structured data, and we are specifically extracting the 'name' field.
for col in json_columns:
    movies_metadata[col] = movies_metadata[col].apply(lambda x: extract_val_json(x, 'name'))

In [11]:
# Display the 5 rows to verify the transformation
movies_metadata.sample(5)    

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
36295,False,NaN,0.0,Documentary,NaN,368835.0,tt4715470,en,Murder Rap: Inside the Biggie and Tupac Murders,Two of the most notorious unsolved cases in th...,...,2015-08-08,0.0,113.0,NaN,Released,NaN,Murder Rap: Inside the Biggie and Tupac Murders,False,7.9,5.0
25840,False,NaN,0.0,"Drama, Music, Romance",NaN,197737.0,tt0031565,en,Let Freedom Ring,A Harvard man (Nelson Eddy) fights a railroad ...,...,1939-02-24,0.0,87.0,English,Released,Thrill-Blasting Drama of Men and Women at the ...,Let Freedom Ring,False,6.0,1.0
3711,False,NaN,0.0,"Action, Comedy, Drama, Romance, Western",NaN,21866.0,tt0080472,en,Bronco Billy,"An idealistic, modern-day cowboy struggles to ...",...,1980-06-11,0.0,116.0,"English, Deutsch",Released,The most outrageous of 'em all.,Bronco Billy,False,5.2,51.0
15442,False,NaN,0.0,Comedy,http://www.puppetryofthepenis.com/,36962.0,tt0313504,en,Puppetry of the Penis: Live at the Forum,This bizarre but compelling cult theatrical hi...,...,2001-01-01,0.0,52.0,English,Released,The Ancient Art of Genital Origami,Puppetry of the Penis: Live at the Forum,False,5.5,3.0
33404,False,NaN,0.0,Drama,http://wherehopegrowsmovie.com/,243352.0,tt3200980,en,Where Hope Grows,Calvin Campbell is a former professional baseb...,...,2014-04-06,1156000.0,95.0,English,Released,AN UNEXPECTED JOURNEY. AN UNEXPECTED FRIENDSHIP.,Where Hope Grows,False,6.8,26.0


### Cleaning NaN and duplicates

In [12]:
# Drop any rows in the DataFrame that are completely identical across all columns.
movies_metadata = movies_metadata.drop_duplicates()

In [13]:
# After processing all the columns, there will be rows where 'id' will be NaN. This indicate the role may have incorrect or incomplete data. These rows will be removed.
movies_metadata.loc[movies_metadata['id'].isna()]
movies_metadata=movies_metadata.dropna(subset=['id']).reset_index(drop=True)

In [14]:
# Each 'id' should represent a unique movie/ row. If the 'id' appears more than once, it means there are data variations.
duplicate_id = movies_metadata.loc[movies_metadata['id'].duplicated(keep=False)] 
len(duplicate_id['id']) # 26 repeated movie ids found

26

In [15]:
# Group by id to find out which columns are causing the 'id' to repeat. 
# Use .nunique() to see the number of unique values per 'id' for each column.
# Result shows that the variations are caused by 'popularity' and 'vote_count' columns.
duplicate_id.groupby('id').nunique().sort_index()

,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
id,,,,,,,,,,,,,,,,,,,,,
4912.0,1,0,1,1,0,1,1,1,1,2,...,1,1,1,1,1,1,1,1,1,1
10991.0,1,1,1,1,1,1,1,1,1,2,...,1,1,1,1,1,1,1,1,1,2
12600.0,1,1,1,1,1,1,1,1,1,2,...,1,1,1,1,1,0,1,1,1,1
13209.0,1,0,1,1,0,1,1,1,1,2,...,1,1,1,1,1,0,1,1,1,1
14788.0,1,0,1,1,1,1,1,1,1,2,...,1,1,1,1,1,0,1,1,1,1
15028.0,1,0,1,1,0,1,1,1,1,2,...,1,1,1,1,1,1,1,1,1,2
22649.0,1,0,1,1,0,1,1,1,1,2,...,1,1,1,1,1,1,1,1,1,1
69234.0,1,0,1,1,0,1,1,1,1,2,...,1,1,1,1,1,0,1,1,1,1
77221.0,1,0,1,1,0,1,1,1,1,2,...,1,1,1,1,1,0,1,1,1,1


In [16]:
# Repeated movie 'id' are caused by variations in popularity and vote_count values. They will be averaged.
movies_metadata['popularity'] = movies_metadata.groupby(['id'])['popularity'].transform('mean')
movies_metadata['vote_count'] = movies_metadata.groupby(['id'])['vote_count'].transform('mean')

In [17]:
# Drop any duplicated rows that are entirely same
movies_metadata=movies_metadata.drop_duplicates()

In [18]:
# Export the data
movies_metadata.to_csv('cleaned/cleaned_movies_metadata.csv')

## Data Cleaning - Movies Keywords dataset

In [19]:
keywords_original = pd.read_csv('original_files/keywords.csv')

In [20]:
# 'keywords' column has JSON-like structure, movie keyword need to be extracted.
keywords_original.head(5)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [21]:
# Both columns ('id' and 'keywords') do not have missing values.
keywords_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


In [22]:
# However there are multiple repeated 'id'. 
keywords_original.loc[keywords_original['id'].duplicated(keep=False)].sort_values(by='id')

,id,keywords
36138,1998,"[{'id': 417, 'name': 'corruption'}, {'id': 612..."
37095,1998,"[{'id': 417, 'name': 'corruption'}, {'id': 612..."
35865,3025,"[{'id': 212, 'name': 'london england'}, {'id':..."
36822,3025,"[{'id': 212, 'name': 'london england'}, {'id':..."
35999,3692,"[{'id': 470, 'name': 'spy'}, {'id': 591, 'name..."
...,...,...
36190,380841,[]
36193,380864,[]
37150,380864,[]
37280,381353,[]


In [23]:
# There are duplicate rows that needs to be dropped. Will be dropped in later steps.
keywords_original.drop_duplicates().info()

<class 'pandas.core.frame.DataFrame'>
Index: 45432 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        45432 non-null  int64 
 1   keywords  45432 non-null  object
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [24]:
movie_keywords = keywords_original.copy()
# Convert 'keywords' column to a JSON-like format if it's a string representation of a list/dict.
movie_keywords['keywords'] = movie_keywords['keywords'].apply(lambda x:  must_be_type(x, 'json'))
# Extract 'name' field for each JSON entry and format as a comma-seperated string.
movie_keywords['keywords'] = movie_keywords['keywords'].apply(lambda x:  extract_val_json(x, 'name'))

# Drop the duplicates
movie_keywords = movie_keywords.drop_duplicates()

In [25]:
movie_keywords

,id,keywords
0,862,"jealousy, toy, boy, friendship, friends, rival..."
1,8844,"board game, disappearance, based on children's..."
2,15602,"fishing, best friend, duringcreditsstinger, ol..."
3,31357,"based on novel, interracial relationship, sing..."
4,11862,"baby, midlife crisis, confidence, aging, daugh..."
...,...,...
46414,439050,tragic love
46415,111109,"artist, play, pinoy"
46416,67758,NaN
46417,227506,NaN


In [26]:
movie_keywords.to_csv('cleaned/movie_keywords.csv')

## Data Cleaning - Credits dataset

In [27]:
credits_original = pd.read_csv('original_files/credits.csv')

In [28]:
# 'cast' and 'crew' columns have JSON-like structure, individual names need to be extracted.
credits_original.head(5)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [29]:
# All columns do not have missing values.
credits_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [30]:
# There are duplicate rows that needs to be dropped. Will be dropped in later steps.
credits_original.drop_duplicates().info()

<class 'pandas.core.frame.DataFrame'>
Index: 45439 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45439 non-null  object
 1   crew    45439 non-null  object
 2   id      45439 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [31]:
movie_credits = credits_original.copy()

# Convert columns to a JSON-like format if it's a string representation of a list/dict.
movie_credits['cast'] = movie_credits['cast'].apply(lambda x:  must_be_type(x, 'json'))
movie_credits['crew'] = movie_credits['crew'].apply(lambda x:  must_be_type(x, 'json'))

# Extract individual 'name' field for each JSON entry and format as a comma-seperated string.
movie_credits['cast'] = movie_credits['cast'].apply(lambda x: extract_val_json(x, 'name'))
movie_credits['crew'] = movie_credits['crew'].apply(lambda x: extract_val_json(x, 'name'))

# Drop the duplicates
movie_credits = movie_credits.drop_duplicates()

In [32]:
movie_credits

,cast,crew,id
0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...","John Lasseter, Joss Whedon, Andrew Stanton, Jo...",862
1,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...","Larry J. Franco, Jonathan Hensleigh, James Hor...",8844
2,"Walter Matthau, Jack Lemmon, Ann-Margret, Soph...","Howard Deutch, Mark Steven Johnson, Mark Steve...",15602
3,"Whitney Houston, Angela Bassett, Loretta Devin...","Forest Whitaker, Ronald Bass, Ronald Bass, Ezr...",31357
4,"Steve Martin, Diane Keaton, Martin Short, Kimb...","Alan Silvestri, Elliot Davis, Nancy Meyers, Na...",11862
...,...,...,...
45471,"Leila Hatami, Kourosh Tahami, Elham Korda","Hamid Nematollah, Hamid Nematollah, Farshad Mo...",439050
45472,"Angel Aquino, Perry Dizon, Hazel Orencio, Joel...","Lav Diaz, Lav Diaz, Dante Perez, Lav Diaz, Lav...",111109
45473,"Erika Eleniak, Adam Baldwin, Julie du Page, Ja...","Mark L. Lester, C. Courtney Joyner, Jeffrey Go...",67758
45474,"Iwan Mosschuchin, Nathalie Lissenko, Pavel Pav...","Yakov Protazanov, Joseph N. Ermolieff",227506


In [33]:
movie_credits.to_csv('cleaned/movie_credits.csv')